In [1]:
%pip install fastapi pydantic toml openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
import re
import tomllib
import functools
from typing import Literal

import ipywidgets as widgets
from IPython.display import display

from dotenv import load_dotenv

load_dotenv()

from rs import Message, Config, chat, skill_check, chat_with_azure
import d20

action_pattern = re.compile(r"(\d+). *(.+)")
skill_pattern = re.compile(r"[\w_]+")
skill_difficulty_pattern = re.compile(r"(easy|medium|hard)")

def next_round(b):
    global current_round, initial_messages
    current_round += 1
    initial_messages = config.initial_messages(current_round, config.metadata)

    print(initial_messages[-1].content)
    display(get_controls(initial_messages[-1]))

def get_user_message_tail():
    count = 0
    for msg in initial_messages:
        if msg.role == "user":
            count += 1

    is_final =  ", FINAL ROUND" if  current_round == len(config.rounds) else ""
    return f"TURN: {count + 1} {is_final}"
    

def do_chat(role:Literal["user", "system", "assistant"], content: str):
    print(f"{content}\n\n")

    initial_messages.append(Message(role=role, content=content))
    msg = chat_with_azure(initial_messages, lambda delta: print(delta, end=""))

    initial_messages.append(msg)
    display(get_controls(msg))


def on_skill(b, skill:str, difficulty:str):
    content = skill_check(skill, difficulty, config.metadata)
    content += f"\n{get_user_message_tail()}"
    do_chat("user", content)

def on_action(b, index:int):
    content = f"I select option {index}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_input(b, input:widgets.Text):
    content = f"{input.value}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_action(b):
    hbox = widgets.HBox(layout=widgets.Layout(width="100%"))
    action_text = widgets.Text(layout=widgets.Layout(width="100%"))
    submit = widgets.Button(description="SUBMIT", layout=widgets.Layout(width="fit-content"))
    submit.on_click(functools.partial(on_custom_input, input=action_text))
    hbox.children += (action_text, submit) 
    display(hbox)

def get_controls(msg:Message) -> widgets.Box:
    dict = msg.dict()
    buttons = []
    if "actions" in dict.keys():
        matches = action_pattern.finditer(dict["actions"])
        for match in matches:
            btn = widgets.Button(description=match.group(1), tooltip=match.group(2), layout=widgets.Layout(width="auto"))
            btn.on_click(functools.partial(on_action, index=int(match.group(1))))
            buttons.append(btn)
    elif "skill" in dict.keys() and "difficulty" in dict.keys():
        match = skill_pattern.match(dict["skill"])
        skill = match.group(0)
        difficulty = dict["difficulty"]
        skill_button = widgets.Button(description=f"{skill.upper()} [{difficulty.upper()}]", layout=widgets.Layout(width="auto"))
        skill_button.on_click(functools.partial(on_skill, skill=skill, difficulty=difficulty))
        buttons.append(skill_button)
    elif "summary" in dict.keys():
        # add summary as a memory to the metadata
        # it will be used in the next round.
        # config.metadata["memories"] = []
        if ("memories" not in config.metadata):
            config.metadata["memories"] = []
            
        config.metadata["memories"].append(dict["summary"])
        next_btn = widgets.Button(description="NEXT ROUND", layout=widgets.Layout(width="auto"))
        next_btn.on_click(next_round)
        buttons.append(next_btn)
    elif "ending" in dict.keys():
        ending_btn = widgets.Button(description="END", layout=widgets.Layout(width="auto"))
        buttons.append(ending_btn)
    
    custom_action_button = widgets.Button(description="CUSTOM")
    custom_action_button.on_click(on_custom_action)
    buttons.append(custom_action_button)

    return widgets.HBox(buttons)

# open config file
config_name = "data/config_beforesunrise_add_rounds.toml"
config: Config | None = None
with open(config_name, "rb") as f:
    data = tomllib.load(f)
    config = Config(**data)

current_round = 8
initial_messages = config.initial_messages(current_round, config.metadata)

print(initial_messages[-1].content)
display(get_controls(initial_messages[-1]))

context: 你和席琳十指紧扣行走在午夜的维也纳街头，宛如一对热恋中的情侣。突然，一个身影出现在我们面前，是一位年迈的神婆。她慈祥地看着你们，“年轻的恋人们，眼里只有彼此，看看你们缘分吧，只花五美元。” “没问题。” 你和席琳伸出手，神婆仔细地端详着你们掌纹，缓缓说道：“很有缘的两个人，一见如故，一见钟情。”你们默契地相视一笑。“但命运捉弄，离别就要来临，可惜有缘无分。年轻人们，珍惜当下吧。”

神婆的话把你们拉回了现实，和席琳快乐的相处让你忘记了明早要飞回洛杉矶。关于这段恋情，你不确定是否能承诺未来。你决定...

actions:
1. 幽默化解，回避现实。
2. 和席琳聊聊未来的计划。
3. 抱紧席琳。


I select option 1. TURN: 1 


context: 你试图用幽默来化解紧张的气氛，对席琳说：“看来我们要开始信命运了，也许我们是上辈子的情人呢。”席琳轻笑，但你看得出她眼中闪过一丝忧虑。你意识到，尽管你想要轻松地对待这一切，但这段关系和未来的不确定性是无法简单通过幽默来逃避的。席琳轻声说：“我知道你想让气氛轻松一些，但我们真的需要谈谈，杰西。”

actions:
1. 坦诚表达你的担忧。
2. 询问席琳的想法和感受。
3. 邀请席琳一起散步，边走边聊。

I select option 1. TURN: 2 


context: 你深吸一口气，决定坦诚地表达你的担忧。“席琳，我刚从一段关系中恢复过来，我害怕太快投入新的感情。而且，我们之间的距离，我在美国，你在法国，这让我更加不确定。”席琳听着，眼中有泪光，但她尽力保持镇定。“我理解你的感受，杰西。我也害怕，害怕这段美好的时光过后，我们会变成陌生人。但我也不想放弃，即使前路未知。”

actions:
1. 提议保持联系，看看未来如何。
2. 讨论可能的解决方案。
3. 坦诚地提出明早就说再见。

I select option 2. TURN: 3 


context: 你们决定坐下来，认真讨论可能的解决方案。“我们可以尝试维持长距离关系，保持沟通，每天视频通话，共享日常。”你建议。席琳点头，“我们也可以轮流访问对方，尽可能缩短这个距离感。”但随着谈话深入，你们逐渐意识到，尽管这些努力听起来很美好，实际操作起来却充满挑战。时间差、生活节奏的不同、甚至文化差异，都可能成为你们关系的障碍。
summary: 杰西和席琳在月光下认真讨论了他们的未来。杰西首先尝试用幽默化解紧张气氛，但最终选择坦诚表达自己的担忧。他们尝试讨论可能的解决方案，但都意识到维持这段异地恋充满挑战。最终，他们达成了一致，珍惜当下，明早就说再见，不再联系，以避免未来的不确定性和可能的痛苦。